In [1]:
import pandas as pd
import scipy as sc
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
data_cleaned = pd.read_csv("data/data_cleaned.csv")
label = data_cleaned["Categorie1"].values

In [3]:
data_cleaned.head()

,Categorie1,Description
0,INFORMATIQUE,batter acer aspir one ion mah noir compatibl b...
1,TELEPHONIE - GPS,coqu rigid bleu lagon alcatel motif drapeau li...
2,TELEPHONIE - GPS,facad coqu cellular lin shckgal minip marqu ag...
3,TELEPHONIE - GPS,coqu meteor tpu nexus
4,TELEPHONIE - GPS,coqu soupl transparent flex motif keep calm an...


In [4]:
features = sc.sparse.load_npz("data/feature_sparse.npz")

In [5]:
features.getrow(0).indices

array([  314,  3258,  4798, 11521, 27233, 32322, 37207, 38199],
      dtype=int32)


## Train test split 

In [6]:
# X:descriptions, Y:categories
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.1, random_state=42)

In [7]:
X_train

<90000x57925 sparse matrix of type '<class 'numpy.int64'>'
	with 955069 stored elements in Compressed Sparse Row format>

In [8]:
X_test

<10000x57925 sparse matrix of type '<class 'numpy.int64'>'
	with 106031 stored elements in Compressed Sparse Row format>

In [9]:
y_train

array(['BIJOUX -  LUNETTES - MONTRES', 'MEUBLE ', 'ELECTROMENAGER', ...,
       'INFORMATIQUE', 'TELEPHONIE - GPS', 'INFORMATIQUE'], dtype=object)

In [10]:
y_test

array(['JOUET (NEW)', 'TELEPHONIE - GPS', 'TELEPHONIE - GPS', ...,
       'TV - VIDEO - SON', 'ADULTE - EROTIQUE ', 'TELEPHONIE - GPS'],
      dtype=object)

## Regression Logistique

In [11]:
lr = LogisticRegression()

In [12]:
ts = time.time()
lr.fit(X_train, y_train)
te = time.time()
print(te-ts)

66.8657579421997


/Users/cecile/opt/anaconda3/envs/Projet4A/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
Y_train_predict = lr.predict(X_train)
lr.score(X_train,y_train)

0.9869777777777777

# Explications

Y_train predict contient les prédictions description par description
ici on voit les prédictions des 3 premiers fichiers.

In [14]:
Y_train_predict[:3]

array(['BIJOUX -  LUNETTES - MONTRES', 'MEUBLE ', 'ELECTROMENAGER'],
      dtype=object)

Vous pouvez retrouver le score a partir de cette prédiction en faisant:
Nombre de descriptions dont la prédiction est juste/Nombre de descriptions total.

Vous voyez que le score est identique à celui calculé par lr.score (qui fait la même chose).

In [15]:
nb_good_prediction = sum(Y_train_predict == y_train)
n_total = len(y_train)
score = nb_good_prediction/n_total
print(score)

0.9869777777777777


Vous pouvez maintenant faire la même chose mais par catégorie seulement. 
Pour cela on se ramène a une prédiction de une catégorie par exemple pour cette catégorie:

In [16]:
#Liste de catégorie unique
unique_catégorie = list(set(y_train))

In [171]:
cat_name = 'JARDIN - PISCINE'
Y_train_predict_cat = Y_train_predict==cat_name #P
y_train_cat = y_train==cat_name

nb_good_prediction = sum(Y_train_predict_cat == y_train_cat)
n_total = len(y_train_cat)
score = nb_good_prediction/n_total
print(score)

0.9991444444444444


Le score (accuracy) est très bon mais il est trompeur. Car finalement nous avons beaucoup d'éléments qui ne sont PAS de cette catégorie. 
Il faut donc regarder un peu plus en détails d'autre score à partir de la table des éléments croisé.s

Vous pouvez voir qu'il y a énormément de VRAI NEGATIF (element prédit faux qui sont effectivement faux) c'est ce qui permet d'avoir un bon score.

In [18]:
A=pd.crosstab(y_train_cat, Y_train_predict_cat, rownames=["Ground True"], colnames=["Prediction"])

Pour avoir une meilleur idée on peut combiner ces quatres score:
* TRUE NEGATIVE : élément faux prédit faux / tout les élement faux
* TRUE POSITIVE : élément vrai prédit vrai / tout les élement vrai
* FALSE NEGATIVE : élément vrai prédit faux / tout les élement vrai
* FALSE POSITIVE :  élément faux prédit vrai/ tout les élement faux

(https://en.wikipedia.org/wiki/F1_score#diagnostic_testing (Partie diagnostic testing)

Le code en bas permet de les calculer (je vous laisse regarder ça)

In [19]:
A

Prediction,False,True
Ground True,,
False,88771,6
True,46,1177


In [20]:
A/A.sum(axis=1)*100

Prediction,False,True
Ground True,,
False,99.993241,0.490597
True,0.051815,96.238757


In [21]:
print('Taux de classification : ', (88771+1177)/(88771+1177+6+46)) 
#idem que score

Taux de classification :  0.9994222222222222


### Comment :

* On se rend compte que l'algo a un taux de classification à 99,94%. 
* Lorsqu'il s'agit de dire que l'élément n'appartient pas à la catégorie Jardin-Piscine, l'algo a 99,99% de chance de ne pas se tromper.
* En revanche, l'algo a 5% de chance de dire que la description n'appartient pas à la catégorie Jardin-Piscine alors que si elle était dans cette catégorie.

## Score par catégorie

In [182]:
cat = list(set(y_train)) #recup toutes les categories

for i in cat :
    cat_name = str(i)
    Y_train_predict_cat = Y_train_predict==cat_name
    y_train_cat = y_train==cat_name
    nb_good_prediction = sum(Y_train_predict_cat == y_train_cat)
    n_data = 0
    for k in y_train_cat:
        if k== True:
            n_data +=1
    n_total = len(y_train_cat)
    score = nb_good_prediction/n_total
    print(cat_name, "    ", n_data, "data") 
    print("Score : ", score, "\n") 
    A=pd.crosstab(y_train_cat, Y_train_predict_cat, rownames=["Ground True"], colnames=["Prediction"])
    print(A/A.sum(axis=1)*100, "\n \n \n")

PHOTO - OPTIQUE      1408 data
Score :  0.9988222222222222 

Prediction       False      True 
Ground True                      
False        99.959364   2.556818
True          0.079014  95.028409 
 
 

TATOUAGE - PIERCING      135 data
Score :  0.9999777777777777 

Prediction        False      True 
Ground True                       
False        100.000000   0.000000
True           0.002226  98.518519 
 
 

INSTRUMENTS DE MUSIQUE      331 data
Score :  0.9998444444444444 

Prediction       False      True 
Ground True                      
False        99.997770   0.604230
True          0.013383  96.374622 
 
 

ADULTE - EROTIQUE       798 data
Score :  0.9988888888888889 

Prediction       False      True 
Ground True                      
False        99.976458   2.631579
True          0.088563  90.100251 
 
 

ELECTROMENAGER      1919 data
Score :  0.9995333333333334 

Prediction       False      True 
Ground True                      
False        99.980700   0.885878
True       

DECO - LINGE - LUMINAIRE      5272 data
Score :  0.9972888888888889 

Prediction       False      True 
Ground True                      
False        99.879615   1.934750
True          0.167595  97.306525 
 
 

MEUBLE       1234 data
Score :  0.9996777777777778 

Prediction       False      True 
Ground True                      
False        99.985355   1.053485
True          0.018025  98.703404 
 
 

HYGIENE - BEAUTE - PARFUM      1221 data
Score :  0.9994777777777778 

Prediction       False      True 
Ground True                      
False        99.983104   1.228501
True          0.036045  97.379197 
 
 

MATERIEL DE BUREAU      77 data
Score :  0.9999444444444444 

Prediction       False      True 
Ground True                      
False        100.00000   0.000000
True           0.00556  93.506494 
 
 



### Comment :

* On se rend compte que le taux de classification pour chaque catégorie est très très élevé, > 99,5%.
* Plus il y a de produit dans une catégorie au sein du set d'apprentissage plus le True - True de cette catégorie est élevé. 

## Reduction de X train

In [24]:
nb = 10000
X_small_train = X_train[0:nb]
y_small_train = y_train[0:nb]

In [25]:
ts = time.time()
lr.fit(X_small_train, y_small_train)
te = time.time()
print(te-ts)

35.03369116783142


In [26]:
Y_small_train_predict = lr.predict(X_small_train)
lr.score(X_small_train,y_small_train)

0.9926

In [27]:
lr.score(X_test,y_test)

0.826

#index_gps = lbl_gps.loc[lbl_gps['Categorie1'] == 'TELEPHONIE - GPS'].index
#features_gps = features[v,:]
#lbl_gps = lbl_gps.reset_index(drop=True)


    ### Grosse erreur: on voulait faire la regression logistique par catégorie au lieu d'obtenir le score par categorie

## Comments:
 
Observations : plus on réduit le nombre de produit dans X_train plus notre score de classification est bon sur ce même set. A l'inverse le taux de classification se dégrade sur l'ensemble test. Cohérent car il y a moins de données pour l'apprentissage. On obtient un score de 82,89% pour des set d'apprentissage et de test de 10 000 produits.

 